In [164]:
import csv
import pprint

# with open("skill_levels.csv", encoding="utf8") as f:
#     skill_table = [x for x in csv.DictReader(f)]

with open("armor_base.csv", encoding="utf8") as f:
    armor_table = [x for x in csv.DictReader(f)]

with open("armor_skills_ext.csv", encoding="utf8") as f:
    armor_skills_table = dict((x["base_name_en"], x) for x in csv.DictReader(f))

with open("charm_base.csv", encoding="utf8") as f:
    charm_table = [x for x in csv.DictReader(f)]

with open("decoration_base.csv", encoding="utf8") as f:
    deco_table = [x for x in csv.DictReader(f)]

print(armor_table[-1])
print(armor_skills_table["Leather Headgear"])
print(charm_table[0])
print(deco_table[0])


{'id': '1470', 'name_en': 'Wyverian Ears α+', 'rarity': '9', 'type': 'head', 'gender': 'both', 'slot_1': '4', 'slot_2': '0', 'slot_3': '0', 'defense_base': '140', 'defense_max': '178', 'defense_augment_max': '198', 'defense_fire': '1', 'defense_water': '1', 'defense_thunder': '1', 'defense_ice': '1', 'defense_dragon': '5'}
{'base_name_en': 'Leather Headgear', 'skill1_name': 'Hunger Resistance', 'skill1_level': '1', 'skill2_name': '', 'skill2_level': ''}
{'id': '1', 'name_en': 'Poison Charm I', 'previous_en': '', 'rarity': '3', 'skill1_name': 'Poison Resistance', 'skill1_level': '1', 'skill2_name': '', 'skill2_level': ''}
{'id': '1', 'name_en': 'Antidote Jewel 1', 'slot': '1', 'rarity': '5', 'skill1_name': 'Poison Resistance', 'skill1_level': '1', 'skill2_name': '', 'skill2_level': '', 'icon_color': 'Violet'}


In [165]:
# join and reformat.
equipment_table = []
for armor in armor_table:
    if int(armor["rarity"]) <= 8:
        continue
    equipment_table.append({
        "name_en": armor["name_en"],
        "rarity": int(armor["rarity"]),
        "type": armor["type"],
        "slot_1": int(armor["slot_1"]),
        "slot_2": int(armor["slot_2"]),
        "slot_3": int(armor["slot_3"]),
        "defense_base": int(armor["defense_base"]),
        "defense_max": int(armor["defense_max"]),
        "defense_augment_max": int(armor["defense_augment_max"]),
        # and whatever ig

        "skill1_name": armor_skills_table[armor["name_en"]]["skill1_name"],
        "skill1_level": int(armor_skills_table[armor["name_en"]]["skill1_level"]) if armor_skills_table[armor["name_en"]]["skill1_name"] != "" else 0,
        "skill2_name": armor_skills_table[armor["name_en"]]["skill2_name"],
        "skill2_level": int(armor_skills_table[armor["name_en"]]["skill2_level"]) if armor_skills_table[armor["name_en"]]["skill2_name"] != "" else 0,

        "1_slots": int(int(armor["slot_1"]) >= 1) + int(int(armor["slot_2"]) >= 1) + int(int(armor["slot_3"]) >= 1),
        "2_slots": int(int(armor["slot_1"]) >= 2) + int(int(armor["slot_2"]) >= 2) + int(int(armor["slot_3"]) >= 2),
        "3_slots": int(int(armor["slot_1"]) >= 3) + int(int(armor["slot_2"]) >= 3) + int(int(armor["slot_3"]) >= 3),
        "4_slots": int(int(armor["slot_1"]) >= 4) + int(int(armor["slot_2"]) >= 4) + int(int(armor["slot_3"]) >= 4),
    })
for charm in charm_table:
    equipment_table.append({
        "name_en": charm["name_en"],
        "rarity": int(charm["rarity"]),
        "type": "charm",
        "slot_1": 0,
        "slot_2": 0,
        "slot_3": 0,
        "defense_base": 0,
        "defense_max": 0,
        "defense_augment_max": 0,
        # and whatever ig

        "skill1_name": charm["skill1_name"],
        "skill1_level": int(charm["skill1_level"]) if charm["skill1_name"] != "" else 0,
        "skill2_name": charm["skill2_name"],
        "skill2_level": int(charm["skill2_level"]) if charm["skill2_name"] != "" else 0,
        
        "1_slots": 0,
        "2_slots": 0,
        "3_slots": 0,
        "4_slots": 0,
    })
for deco in deco_table:
    equipment_table.append({
        "name_en": deco["name_en"],
        "rarity": int(deco["rarity"]),
        "type": "deco",
        "slot_1": -int(deco["slot"]), # hack. makes sense semantically.
        "slot_2": 0,
        "slot_3": 0,
        "defense_base": 0,
        "defense_max": 0,
        "defense_augment_max": 0,
        # and whatever ig

        "skill1_name": deco["skill1_name"],
        "skill1_level": int(deco["skill1_level"]) if deco["skill1_name"] != "" else 0,
        "skill2_name": deco["skill2_name"],
        "skill2_level": int(deco["skill2_level"]) if deco["skill2_name"] != "" else 0,

        "1_slots": -1 * int(int(deco["slot"]) >= 1), # hack. makes sense semantically.
        "2_slots": -1 * int(int(deco["slot"]) >= 2),
        "3_slots": -1 * int(int(deco["slot"]) >= 3),
        "4_slots": -1 * int(int(deco["slot"]) >= 4),
    })
# TODO: Weapons and their deco slots. Maybe should be pinned instead.
for weapon in [{"slot_1": 4, "slot_2": 0, "slot_3": 0}]:
    equipment_table.append({
        "name_en": "Your Weapon Here (Adjust slots)",
        "rarity": None,
        "type": "weapon",
        "slot_1": weapon["slot_1"],
        "slot_2": weapon["slot_2"],
        "slot_3": weapon["slot_3"],
        "defense_base": 0,
        "defense_max": 0,
        "defense_augment_max": 0,
        # and whatever ig

        "skill1_name": "",
        "skill1_level": 0,
        "skill2_name": "",
        "skill2_level": 0,

        "1_slots": int(int(weapon["slot_1"]) >= 1) + int(int(weapon["slot_2"]) >= 1) + int(int(weapon["slot_3"]) >= 1),
        "2_slots": int(int(weapon["slot_1"]) >= 2) + int(int(weapon["slot_2"]) >= 2) + int(int(weapon["slot_3"]) >= 2),
        "3_slots": int(int(weapon["slot_1"]) >= 3) + int(int(weapon["slot_2"]) >= 3) + int(int(weapon["slot_3"]) >= 3),
        "4_slots": int(int(weapon["slot_1"]) >= 4) + int(int(weapon["slot_2"]) >= 4) + int(int(weapon["slot_3"]) >= 4),
    })

In [166]:
skill_list = tuple(sorted(set(equipment["skill1_name"] for equipment in equipment_table if equipment["skill1_name"] != "").union(set(equipment["skill2_name"] for equipment in equipment_table if equipment["skill2_name"] != ""))))
# for letter in "ABCDEFGHIJKLMNOPQRSTUVWXYZ":
#     out = tuple(skill for skill in skill_list if skill[0] == letter)
#     if len(out) > 0:
#         print(out)

desired_skills = {
    "Artillery": 5,
    "Capacity Boost": 1,
    "Focus": 2,

    "Evade Extender": 3,
    "Health Boost": 3,
    # "Divine Blessing": 3,
    # "Quick Sheath": 3,
    # "Slinger Capacity": 3,
    # "Stun Resistance": 1,
    # "Free Meal": 1,
    "Resuscitate": 1,
    # "Wide-Range": 5,
    # "Free Meal": 3,
    # "Offensive Guard": 1,
    # "Critical Boost": 1,
    "Razor Sharp/Spare Shot": 1,
    "Clutch Claw Boost": 1
}

assert all(desired in skill_list for desired in desired_skills)

In [167]:
deco_list = tuple(sorted(set(deco["name_en"] for deco in deco_table)))
# Typing is faster than copy-pasting but in case you want it.
# for level in range(1, 5):
#     for letters in ("ABCDE", "FGHIJ", "KLMNO", "PQRST", "UVWXYZ"):
#         out = tuple(deco for deco in deco_list if deco[0] in letters and int(deco[-1]) == level)
#         if len(out) > 0:
#             print(out)

owned_decos = {
    "Miasma Jewel+ 4": 1,
    "Expert Jewel+ 4": 1,
    "Wind Resist/Protection Jewel 4": 1,
    "Steadfast Jewel+ 4": 1,
    "Brace/Vitality Jewel 4": 1,
    "Crisis/Attack Jewel 4": 1,
    "Guardian/Attack Jewel 4": 1,
    "Flight/Expert Jewel 4": 1,
    "Crisis/Expert Jewel 4": 1,
    "Sheath/Expert Jewel 4": 1,
    "Stonethrower/Expert Jewel 4": 1,
    "Fortitude/Handicraft Jewel 4": 1,
    "Jumping/Physique Jewel 4": 1,
    "Slider/Evasion Jewel 4": 1,
    "Stonethrower/Evasion Jewel 4": 2,
    "Draw/Maintenance Jewel 4": 1,
    "Enhancer/Maintenance Jewel 4": 1,
    "Challenger/Maintenance Jewel 4": 1,
    "Flawless/Maintenance Jewel 4": 1,

    "Earplug Jewel 3": 1,
    "Shaver Jewel 3": 1,

    # "Magazine Jewel 2": 1,
    "Footing Jewel 2": 7,
    "Tenderizer Jewel 2": 1,
    "Evasion Jewel 2": 1,
    "Jumping Jewel 2": 2,
    "Sprinter Jewel 2": 1,
    "Warming Jewel 2": 1,

    "Artillery Jewel 1": 1,
    "Friendship Jewel 1": 4,
    "Grinder Jewel 1": 1,
    "Sheath Jewel 1": 1,
    "Steadfast Jewel 1": 3,
    "Stonethrower Jewel 1": 3,
    "Miasma Jewel 1": 1,
    "Attack Jewel 1": 1,
    "Expert Jewel 1": 1,
    "Vitality Jewel 1": 3,
}
# to ignore, uncomment. but tbh itll just shove everything into decos.
# owned_decos = dict((x, 1000) for x in deco_list)

assert all(owned in deco_list for owned in owned_decos), next(owned for owned in owned_decos if owned not in deco_list)

In [168]:
armor_list = tuple(sorted(set(armor["name_en"] for armor in armor_table)))
# don't print this out, just check below and refine.

# by substring, to filter out entire sets.
exclude_at_most = {
    "Dragon", "Escadora", "Safi",
    "Golden", "Brachydium", "Kulve", "Kushala", "γ", "Rex Roar", "Grand God", "Empress", "Ruinous", "Frostfang", "Shara", "Guildwork", "Stygian",
    "Silver Sol", "Azure Age",
    "Guild Palace",
    "Demonlord",
}

# assert(all((pinned in equipment_list) for pinned in pin_at_least))
assert(all(any(equip["name_en"].find(excluded) != -1 for equip in equipment_table) for excluded in exclude_at_most))

In [169]:
import numpy
import random

# maximize this (or minimize)
defense_row = numpy.fromiter((equipment["defense_base"] for equipment in equipment_table), int)

# there must be exactly one of each
head_row = numpy.fromiter((equipment["type"] == "head" for equipment in equipment_table), int)
chest_row = numpy.fromiter((equipment["type"] == "chest" for equipment in equipment_table), int)
arms_row = numpy.fromiter((equipment["type"] == "arms" for equipment in equipment_table), int)
waist_row = numpy.fromiter((equipment["type"] == "waist" for equipment in equipment_table), int)
legs_row = numpy.fromiter((equipment["type"] == "legs" for equipment in equipment_table), int)
charm_row = numpy.fromiter((equipment["type"] == "charm" for equipment in equipment_table), int)
weapon_row = numpy.fromiter((equipment["type"] == "weapon" for equipment in equipment_table), int)

equipment_matrix = numpy.vstack((head_row, chest_row, arms_row, waist_row, legs_row, charm_row, weapon_row))
equipment_upper_bound = numpy.ones(7)
equipment_lower_bound = numpy.ones(7)

# unused slots cannot be negative. (there cannot be more decos than there are slots)
# by setup, a 3 slot deco takes a 3 slot spot, a 2 slot spot, and a 1 slot spot.
# if the armor had a 4 slot, its left untaken, but it would be unusable without the 3 slot, the 2 slot, the 1 slot.
# describe this nicer.
unused_4_slots = numpy.fromiter((equipment["4_slots"] for equipment in equipment_table), int)
unused_3_slots = numpy.fromiter((equipment["3_slots"] for equipment in equipment_table), int)
unused_2_slots = numpy.fromiter((equipment["2_slots"] for equipment in equipment_table), int)
unused_1_slots = numpy.fromiter((equipment["1_slots"] for equipment in equipment_table), int)
deco_slot_matrix = numpy.vstack((unused_4_slots, unused_3_slots, unused_2_slots, unused_1_slots))
deco_slot_lower_bound = numpy.zeros(4)

# there must be the user selected skills
def get_skill_level(equipment, skill):
    if equipment["skill1_name"] == skill:
        return equipment["skill1_level"]
    if equipment["skill2_name"] == skill:
        return equipment["skill2_level"]
    return 0

skill_matrix = numpy.vstack(
    tuple(
        numpy.fromiter((get_skill_level(equipment, skill) for equipment in equipment_table), int)
        for skill in skill_list
    )
)
skill_lower_bound = numpy.fromiter((desired_skills[skill] if skill in desired_skills else 0 for skill in skill_list), int)

# decos must be owned by the user
deco_matrix = numpy.vstack(
    tuple(
        numpy.fromiter((equipment["name_en"] == deco for equipment in equipment_table), int)
        for deco in deco_list
    )
)
deco_upper_bound = numpy.fromiter((owned_decos[deco] if deco in owned_decos else 0 for deco in deco_list), int)

# all excluded equipment must be bound.
exclude_matrix = numpy.asmatrix(numpy.fromiter((any(equipment["name_en"].find(excluded) != -1 for excluded in exclude_at_most) for equipment in equipment_table), int))
exclude_upper_bound = numpy.zeros(1)

hardcoded_zorah_magdalos_set_please_remove_me_matrix = numpy.asmatrix(numpy.fromiter((equipment["name_en"].find("Zorah") != -1 for equipment in equipment_table), int))
hardcoded_zorah_magdalos_set_please_remove_me_lower_bound = numpy.fromiter([3], int)

In [170]:
from scipy import optimize

res = optimize.linprog(
    c=-defense_row,
    A_ub=numpy.concatenate((equipment_matrix, -equipment_matrix, -deco_slot_matrix, -skill_matrix, deco_matrix, exclude_matrix, -hardcoded_zorah_magdalos_set_please_remove_me_matrix)),
    b_ub=numpy.concatenate((equipment_upper_bound, -equipment_lower_bound, -deco_slot_lower_bound, -skill_lower_bound, deco_upper_bound, exclude_upper_bound, -hardcoded_zorah_magdalos_set_please_remove_me_lower_bound)),
    bounds=(0, 7),
    integrality=1,
)

def interpret(res, integrality = True):
    if res.status != 0:
        print("No solution :(")
        return

    output = tuple((round(count) if integrality else count, equipment) for count, equipment in zip(res.x, equipment_table) if count > 0)
    
    for count, equipment in output:
        print(count, equipment["name_en"], "(", equipment["skill1_name"], equipment["skill2_name"], ")")

    skills = dict()
    for count, equipment in output:
        if equipment["skill1_name"] != "":
            if equipment["skill1_name"] not in skills:
                skills[equipment["skill1_name"]] = 0
            skills[equipment["skill1_name"]] += int(equipment["skill1_level"]) * count
        if equipment["skill2_name"] != "":
            if equipment["skill2_name"] not in skills:
                skills[equipment["skill2_name"]] = 0
            skills[equipment["skill2_name"]] += int(equipment["skill2_level"]) * count
    pprint.pprint(skills)

    print(f"defense: {sum(count * equipment['defense_base'] for count, equipment in output)}")
    
    slot_counts = [0, 0, 0, 0]
    used_slots = [0, 0, 0, 0]
    for count, equipment in output:
        if equipment["1_slots"] > 0:
            slot_counts[3] += count * (equipment["4_slots"]) 
            slot_counts[2] += count * (equipment["3_slots"] - equipment["4_slots"]) 
            slot_counts[1] += count * (equipment["2_slots"] - equipment["3_slots"]) 
            slot_counts[0] += count * (equipment["1_slots"] - equipment["2_slots"]) 
        else:
            used_slots[3] -= count * (equipment["4_slots"])
            used_slots[2] -= count * (equipment["3_slots"] - equipment["4_slots"])
            used_slots[1] -= count * (equipment["2_slots"] - equipment["3_slots"])
            used_slots[0] -= count * (equipment["1_slots"] - equipment["2_slots"])
    print("slot counts:", slot_counts)
    print("used slots:", used_slots)

interpret(res)

1 Dodogama Greaves β+ ( Capacity Boost  )
1 Rathalos Coil α+ ( Fire Attack Focus )
1 Zorah Headgear α+ ( Flinch Free Artillery )
1 Zorah Hide α+ ( Tremor Resistance Artillery )
1 Zorah Claws β+ ( Flinch Free  )
1 Razor Sharp Charm ( Razor Sharp/Spare Shot  )
2 Jumping Jewel 2 ( Evade Extender  )
2 Vitality Jewel 1 ( Health Boost  )
1 Artillery Jewel 1 ( Artillery  )
1 Jumping/Physique Jewel 4 ( Evade Extender Constitution )
1 Crisis/Expert Jewel 4 ( Resuscitate Critical Eye )
1 Brace/Vitality Jewel 4 ( Flinch Free Health Boost )
1 Shaver Jewel 3 ( Clutch Claw Boost  )
1 Your Weapon Here (Adjust slots) (   )
{'Artillery': 5,
 'Capacity Boost': 1,
 'Clutch Claw Boost': 1,
 'Constitution': 1,
 'Critical Eye': 1,
 'Evade Extender': 3,
 'Fire Attack': 2,
 'Flinch Free': 4,
 'Focus': 2,
 'Health Boost': 3,
 'Razor Sharp/Spare Shot': 1,
 'Resuscitate': 1,
 'Tremor Resistance': 2}
defense: 740
slot counts: [3, 3, 1, 3]
used slots: [3, 2, 1, 3]


In [171]:
import random
res = optimize.linprog(
    # c=-numpy.fromiter((random.random() for _ in defense_row), float), # random weights
    c=defense_row, # minimize(!) defense
    A_ub=numpy.concatenate((equipment_matrix, -equipment_matrix, -deco_slot_matrix, -skill_matrix, deco_matrix, exclude_matrix)),
    b_ub=numpy.concatenate((equipment_upper_bound, -equipment_lower_bound, -deco_slot_lower_bound, -skill_lower_bound, deco_upper_bound, exclude_upper_bound)),
    bounds=(0, 7),
    integrality=1,
)

interpret(res)

1 Bone Mail β+ ( Attack Boost  )
1 Kestodon Guards α+ ( Affinity Sliding Focus )
1 Dodogama Coil β+ ( Artillery  )
1 Anja Greaves β+ ( Artillery  )
1 Brachydios Helm α+ ( Agitator Capacity Boost )
1 Razor Sharp Charm ( Razor Sharp/Spare Shot  )
2 Jumping Jewel 2 ( Evade Extender  )
3 Vitality Jewel 1 ( Health Boost  )
1 Artillery Jewel 1 ( Artillery  )
1 Jumping/Physique Jewel 4 ( Evade Extender Constitution )
1 Crisis/Expert Jewel 4 ( Resuscitate Critical Eye )
1 Shaver Jewel 3 ( Clutch Claw Boost  )
1 Your Weapon Here (Adjust slots) (   )
{'Affinity Sliding': 1,
 'Agitator': 2,
 'Artillery': 5,
 'Attack Boost': 1,
 'Capacity Boost': 1,
 'Clutch Claw Boost': 1,
 'Constitution': 1,
 'Critical Eye': 1,
 'Evade Extender': 3,
 'Focus': 2,
 'Health Boost': 3,
 'Razor Sharp/Spare Shot': 1,
 'Resuscitate': 1}
defense: 624
slot counts: [5, 0, 1, 4]
used slots: [4, 2, 1, 2]


In [172]:
letter = None
for new_skill in skill_list:
    new_skill_vec = numpy.fromiter((skill == new_skill for skill in skill_list), int)

    res = optimize.linprog(
        # maximize equipment with skill. higher skill values prioritized
        c=-new_skill_vec @ skill_matrix,
        A_ub=numpy.concatenate((equipment_matrix, -equipment_matrix, -deco_slot_matrix, -skill_matrix, deco_matrix, exclude_matrix, -hardcoded_zorah_magdalos_set_please_remove_me_matrix)),
        b_ub=numpy.concatenate((equipment_upper_bound, -equipment_lower_bound, -deco_slot_lower_bound, -skill_lower_bound, deco_upper_bound, exclude_upper_bound, -hardcoded_zorah_magdalos_set_please_remove_me_lower_bound)),
        bounds=(0, 7),
        integrality=1, # do not set to zero!
    )

    if res.status == 0 and round(-res.fun) != 0:
        if new_skill[0] != letter:
            if letter != None:
                print()
            letter = new_skill[0]
        print((new_skill, round(-res.fun)), end=", ")

('Affinity Sliding', 2)

, ('Agitator', 1), ('Airborne', 1), ('Artillery', 6), ('Attack Boost', 3), 
('Blast Attack', 1), 
('Capacity Boost', 1), ('Clutch Claw Boost', 1), ('Coldproof', 1), ('Constitution', 1), ('Critical Draw', 1), ('Critical Eye', 4), 
('Defense Boost', 2), ('Divine Blessing', 1), 
('Earplugs', 2), ('Effluvia Resistance', 3), ('Evade Extender', 3), ('Evade Window', 2), 
('Fire Attack', 2), ('Flinch Free', 4), ('Focus', 2), ('Fortify', 1), ('Free Meal', 1), 
('Handicraft', 1), ('Health Boost', 4), 
('Marathon Runner', 1), 
('Offensive Guard', 1), 
('Peak Performance', 1), ('Power Prolonger', 1), 
('Quick Sheath', 2), 
('Razor Sharp/Spare Shot', 1), ('Resuscitate', 2), 
('Slinger Capacity', 3), ('Speed Sharpening', 1), ('Stun Resistance', 3), 
('Tool Specialist', 1), ('Tremor Resistance', 5), 
('Water Attack', 1), ('Weakness Exploit', 1), ('Wide-Range', 3), ('Windproof', 1), 